In [1]:
import pandas as pd

## DATA CLEANING

In [2]:
df = pd.read_excel('Data/Raw Data/uci_online_retail.xlsx')

In [3]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


## ==== InvoiceNo ====

- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.

In [4]:
colName = 'InvoiceNo'

# View the rows that are not valid digits.

temp = pd.DataFrame(df.loc[~df[colName].astype(str).str.isdigit(), colName].unique())
#1.take colName column
#2.change to string datatype
#3.use isdigit() function
#4.filter df on the inverse of this condition using loc
#5.get unique values using unique()
#5.convert this array data structure into DataFrame data structure

temp.columns = [colName]
#6.rename column

temp.sort_values(by=colName)
#7.sort values by ascending

,InvoiceNo
2388,A563185
2389,A563186
2390,A563187
0,C536379
1,C536383
...,...
3834,C581484
3835,C581490
3836,C581499
3837,C581568


In [5]:
temp = df[~df[colName].astype('str').str.contains('C') & ~df[colName].astype('str').str.isdigit()]
temp
#1.convert column to string
#2.search for strings that contain C
#3.filter dataframe on the inverse of this condition
#4.AND with the previous condition we wrote

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


In [6]:
# there's our culprit - the adjustment of bad debt. 
# Hypothesis: the corresponding bad debt source is the listed 6 digit invoice number less the A
# Let's test this out

temp2 = df.iloc[0:0].copy()
#1.create empty dataframe with same column names as original

for invoice in temp[colName].str[1:]:
    temp2.append(df[df[colName] == invoice])
    #append to this empty dataframe any rows that match the stripped invoice number
    
temp2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [7]:
# no match....
# since these are not customer transactions, nor are they tagged to any existing transaction,
# let's just remove them

if colName == 'InvoiceNo':
    df = df.drop(temp.index)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [8]:
# Hypothesis: The 7 digit StockCodes that are Cancelled have a a 6 digit equivalent,
# indicating the invoice the cancellation corresponds to

# Do a left join of the 7 digit-ers with the 6 digit-ers, 
# on a new column that extracts the last 6 digits of the 7 digit-ers

tempL = df[df[colName].astype('str').str.len() == 7]
#1.get 7 digit-ers
tempL[colName+'Trunc'] = tempL[colName].astype('str').str[1:].astype('int')
#2.extract last 6 digits, this will be the join column. 
#Important: must be same datatype as the other column we are joining on

tempR = df[df[colName].astype('str').str.len() == 6]
tempR[colName+'Trunc'] = tempR[colName].astype('int')
#3.create right table

pd.merge(tempL, tempR, left_on=(colName+'Trunc'), right_on=colName, how='left')

C:\Users\bryan\AppData\Local\Temp/ipykernel_6756/1951423686.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempL[colName+'Trunc'] = tempL[colName].astype('str').str[1:].astype('int')
C:\Users\bryan\AppData\Local\Temp/ipykernel_6756/1951423686.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempR[colName+'Trunc'] = tempR[colName].astype('int')


,InvoiceNo_x,StockCode_x,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID_x,Country_x,InvoiceNoTrunc_x,InvoiceNo_y,StockCode_y,Description_y,Quantity_y,InvoiceDate_y,UnitPrice_y,CustomerID_y,Country_y,InvoiceNoTrunc_y
0,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom,536379,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,536383,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,536391,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,536391,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,536391,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9283,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom,581490,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
9284,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,581499,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
9285,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,581568,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
9286,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,581569,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [9]:
# our hypothesis is false. 
# we will circle back to this after cleaning the other columns

## // InvoiceNo is done. //

## ==== Quantity ==== 

- Quantity: The quantities of each product (item) per transaction. Numeric.


In [10]:
colName = 'Quantity'

df[[colName]].describe()

,Quantity
count,541906.000000
mean,9.552297
std,218.081761
min,-80995.000000
25%,1.000000
50%,3.000000
75%,10.000000
max,80995.000000


In [11]:
# min should not be negative; in fact all quantities should be positive in theory
df[df[colName]<=0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [12]:
# some of these are the cancellations. we ignore them and circle back to them later
temp = df[(~df['InvoiceNo'].astype('str').str.contains('C')) & (df[colName]<=0)]
temp

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.0,NaN,United Kingdom
4347,536764,84952C,NaN,-38,2010-12-02 14:42:00,0.0,NaN,United Kingdom
7188,536996,22712,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7189,536997,22028,NaN,-20,2010-12-03 15:30:00,0.0,NaN,United Kingdom
7190,536998,85067,NaN,-6,2010-12-03 15:30:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,2011-12-07 18:36:00,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,2011-12-07 18:38:00,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,2011-12-07 18:38:00,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,2011-12-08 09:56:00,0.0,NaN,United Kingdom


In [13]:
temp['CustomerID'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: CustomerID, dtype: float64

In [14]:
temp['Description'].unique()

array([nan, '?', 'check', 'damages', 'faulty', 'Dotcom sales',
       'reverse 21/5/10 adjustment', 'mouldy, thrown away.', 'counted',
       'Given away', 'Dotcom', 'label mix up', 'samples/damages',
       'thrown away', 'incorrectly made-thrown away.', 'showroom', 'MIA',
       'Dotcom set', 'wrongly sold as sets', 'Amazon sold sets',
       'dotcom sold sets', 'wrongly sold sets', '? sold as sets?',
       '?sold as sets?', 'Thrown away.', 'damages/display',
       'damaged stock', 'broken', 'throw away', 'wrong barcode (22467)',
       'wrong barcode', 'barcode problem', '?lost',
       "thrown away-can't sell.", "thrown away-can't sell", 'damages?',
       're dotcom quick fix.', "Dotcom sold in 6's", 'sold in set?',
       'cracked', 'sold as 22467', 'Damaged',
       'mystery! Only ever imported 1800',
       'MERCHANT CHANDLER CREDIT ERROR, STO', 'POSSIBLE DAMAGES OR LOST?',
       'damaged', 'DAMAGED', 'Display', 'Missing', 'wrong code?',
       'wrong code', 'adjust', 'crush

In [15]:
# these rows mostly consist of inventory upkeep. 
# a rare few are related to sales, but for simplicity we shall just remove them

if colName == 'Quantity':
    df = df.drop(temp.index)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [16]:
df[[colName]].describe()

,Quantity
count,540570.000000
mean,9.958755
std,216.230153
min,-80995.000000
25%,1.000000
50%,3.000000
75%,10.000000
max,80995.000000


## // Quantity is done. //

## ==== InvoiceDate ==== 

- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.


In [17]:
# seems good
df['InvoiceDate'].describe(datetime_is_numeric=True)

count                           540570
mean     2011-07-04 14:42:30.409234432
min                2010-12-01 08:26:00
25%                2011-03-28 11:34:00
50%                2011-07-20 11:07:00
75%                2011-10-19 11:41:00
max                2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

## // InvoiceDate is done. //

## ==== UnitPrice ====

- UnitPrice: Unit price. Numeric, Product price per unit in sterling.


In [18]:
colName = 'UnitPrice'

df[colName].describe()

count    540570.000000
mean          4.642999
std          93.307581
min           0.000000
25%           1.250000
50%           2.080000
75%           4.130000
max       38970.000000
Name: UnitPrice, dtype: float64

In [19]:
# prie shouldn't be zero
df.loc[df[colName] == 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535334,581211,22142,check,14,2011-12-07 18:36:00,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom
538504,581406,46000M,POLYESTER FILLER PAD 45x45cm,240,2011-12-08 13:58:00,0.0,NaN,United Kingdom
538505,581406,46000S,POLYESTER FILLER PAD 40x40cm,300,2011-12-08 13:58:00,0.0,NaN,United Kingdom


In [20]:
# drop those with no price and customerID, because those are untrackable already
temp = df.loc[(df[colName] == 0) & (df['CustomerID'].isna())]

if colName == 'UnitPrice':
    df = df.drop(temp.index)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [21]:
# these have a product and customer, but no price.
# let's try to fill in the missing prices

missingPrices = df.loc[(df[colName] == 0)]
missingPricesIndexes = missingPrices.index
missingPrices

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany
33576,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560.0,United Kingdom
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911.0,EIRE
47068,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
47070,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
56674,541109,22168,ORGANISER WOOD ANTIQUE WHITE,1,2011-01-13 15:10:00,0.0,15107.0,United Kingdom
86789,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
130188,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,2011-03-23 10:25:00,0.0,13239.0,United Kingdom
139453,548318,22055,MINI CAKE STAND HANGING STRAWBERY,5,2011-03-30 12:45:00,0.0,13113.0,United Kingdom
145208,548871,22162,HEART GARLAND RUSTIC PADDED,2,2011-04-04 14:42:00,0.0,14410.0,United Kingdom


In [22]:
# Obtain the average price of each product

tempR = df[['Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', ]]
tempR = tempR[tempR['UnitPrice'] != 0]
#1.unit price should not be zero
tempR = tempR.groupby(by=['Description'], as_index=False).sum()
#2.group by product (as_index=False is important!!)
tempR['AvgPrice'] = tempR['UnitPrice'] / tempR['Quantity']
#3.calculate average price
tempR = tempR[['Description', 'AvgPrice']]
#4.drop all other columns
tempR

,Description,AvgPrice
0,4 PURPLE FLOCK DINNER CANDLES,0.699097
1,50'S CHRISTMAS GIFT BAG LARGE,0.096853
2,DOLLY GIRL BEAKER,0.110858
3,I LOVE LONDON MINI BACKPACK,1.043188
4,I LOVE LONDON MINI RUCKSACK,4.150000
...,...,...
4036,ZINC T-LIGHT HOLDER STARS SMALL,0.051933
4037,ZINC TOP 2 DOOR WOODEN SHELF,35.628750
4038,ZINC WILLIE WINKIE CANDLE STICK,0.101752
4039,ZINC WIRE KITCHEN ORGANISER,5.295667


In [23]:
# Merge the average prices with missingPrices dataframe

missingPrices = pd.merge(missingPrices, tempR, left_on='Description', right_on='Description', how='left')
#1.left join
missingPrices[colName] = missingPrices['AvgPrice']
#2.update 0 values with joined value
missingPrices = missingPrices.drop(labels='AvgPrice',axis=1)
#3.drop that column from the join
missingPrices

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,3.631625,12647.0,Germany
1,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,1.153291,16560.0,United Kingdom
2,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,2.331486,14911.0,EIRE
3,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.408109,13081.0,United Kingdom
4,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.277685,13081.0,United Kingdom
5,541109,22168,ORGANISER WOOD ANTIQUE WHITE,1,2011-01-13 15:10:00,5.551154,15107.0,United Kingdom
6,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.114907,17560.0,United Kingdom
7,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,2011-03-23 10:25:00,0.221615,13239.0,United Kingdom
8,548318,22055,MINI CAKE STAND HANGING STRAWBERY,5,2011-03-30 12:45:00,0.355945,13113.0,United Kingdom
9,548871,22162,HEART GARLAND RUSTIC PADDED,2,2011-04-04 14:42:00,1.199644,14410.0,United Kingdom


In [24]:
if colName == 'UnitPrice':
    df = df.drop(missingPricesIndexes)
    df = pd.concat([df, missingPrices])
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550000,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750000,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
35,577129,22464,HANGING METAL HEART LANTERN,4,2011-11-17 19:52:00,0.215980,15602.0,United Kingdom
36,577168,M,Manual,1,2011-11-18 10:42:00,67.885711,12603.0,Germany
37,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,5.633640,12444.0,Norway
38,577696,M,Manual,1,2011-11-21 11:57:00,67.885711,16406.0,United Kingdom


## // UnitPrice is done. //

## ==== Country ====

- Country: Country name. Nominal, the name of the country where each customer resides.



In [25]:
colName = 'Country'

df[colName].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

## // Country is done. //

## ==== CustomerID ====

- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.


In [26]:
colName = 'CustomerID'

df[colName].describe()

count    406829.000000
mean      15287.690570
std        1713.600303
min       12346.000000
25%       13953.000000
50%       15152.000000
75%       16791.000000
max       18287.000000
Name: CustomerID, dtype: float64

In [27]:
temp = df[df[colName].isna()]
temp 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [28]:
# because our purpose is to obtain CLV, we NEED the customerID column to be valid so that we can group by customer.
# hence, we have to remove these columns

if colName == 'CustomerID':
    df = df.drop(temp.index)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550000,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750000,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
35,577129,22464,HANGING METAL HEART LANTERN,4,2011-11-17 19:52:00,0.215980,15602.0,United Kingdom
36,577168,M,Manual,1,2011-11-18 10:42:00,67.885711,12603.0,Germany
37,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,5.633640,12444.0,Norway
38,577696,M,Manual,1,2011-11-21 11:57:00,67.885711,16406.0,United Kingdom


## // CustomerID is done. //

## ==== StockCode ==== 

- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.


In [29]:
colName = 'StockCode'

# Let's first view StockCodes that don't have 5 digits
df[df[colName].astype('str').str.len() != 5].sort_values(by='StockCode', ascending=True)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
140929,548491,10123C,HEARTS WRAPPING TAPE,1,2011-03-31 13:14:00,0.65,14064.0,United Kingdom
109611,545655,10123C,HEARTS WRAPPING TAPE,3,2011-03-04 13:58:00,0.65,14670.0,United Kingdom
5451,536863,10123C,HEARTS WRAPPING TAPE,1,2010-12-03 11:19:00,0.65,17967.0,United Kingdom
12492,537382,10124A,SPOTS ON RED BOOKCOVER TAPE,4,2010-12-06 13:13:00,0.42,16710.0,United Kingdom
84458,543397,10124A,SPOTS ON RED BOOKCOVER TAPE,3,2011-02-08 10:32:00,0.42,17859.0,United Kingdom
...,...,...,...,...,...,...,...,...
313193,564471,POST,POSTAGE,1,2011-08-25 12:26:00,18.00,12583.0,France
313215,564475,POST,POSTAGE,1,2011-08-25 12:50:00,18.00,12500.0,Germany
313233,564476,POST,POSTAGE,1,2011-08-25 12:57:00,40.00,12610.0,Italy
312087,564337,POST,POSTAGE,1,2011-08-24 14:40:00,18.00,12649.0,Germany


In [30]:
# Hypothesis: Those 6 digit StockCodes are to indicate variants of a base product. 
# Let's see if these 6 digit-ers have a 5 digit equivalent

# Do a left join of the 6 digit-ers with the 5 digit-ers, 
# on a new column that extracts the first 5 digits of the 6 digit-ers

tempL = df[df[colName].astype('str').str.len() == 6][['StockCode', 'Description', 'Quantity']]
#1.get 6 digit-ers
#2.get the 3 relevant columns
tempL[colName+'Trunc'] = tempL[colName].str[:5].astype('int')
#3.extract first 5 digits, this will be the join column. 
#Important: must be same datatype as the other column we are joining on

tempR = df[df[colName].astype('str').str.len() == 5][['StockCode', 'Description', 'Quantity']]
tempR[colName] = tempR[colName].astype('int')
#4.create right table

pd.merge(tempL, tempR, left_on=(colName+'Trunc'), right_on=colName, how='left')

,StockCode_x,Description_x,Quantity_x,StockCodeTrunc,StockCode_y,Description_y,Quantity_y
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,85123,NaN,NaN,NaN
1,84406B,CREAM CUPID HEARTS COAT HANGER,8,84406,NaN,NaN,NaN
2,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,84029,NaN,NaN,NaN
3,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,84029,NaN,NaN,NaN
4,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,85123,NaN,NaN,NaN
...,...,...,...,...,...,...,...
492048,85099C,JUMBO BAG BAROQUE BLACK WHITE,10,85099,NaN,NaN,NaN
492049,84993A,75 GREEN PETIT FOUR CASES,2,84993,NaN,NaN,NaN
492050,85049A,TRADITIONAL CHRISTMAS RIBBONS,1,85049,NaN,NaN,NaN
492051,85049E,SCANDINAVIAN REDS RIBBONS,2,85049,NaN,NaN,NaN


In [31]:
# No match...hence, we cannot merge these products directly.
# Let's observe what different products belong to each base StockCode

from collections import defaultdict
d1 = defaultdict(set)
#1.dictionary of sets

for i in range(len(tempL)):
    d1[tempL['StockCodeTrunc'].iloc[i]].add(tempL['Description'].iloc[i])
    #for every row in dataframe, add the item description to the set (value).
    #the corresponding key is the base StockCode

d = dict((k, tuple(v)) for k, v in d1.items())
d

{85123: ('CREAM HANGING HEART T-LIGHT HOLDER',
  'WHITE HANGING HEART T-LIGHT HOLDER'),
 84406: ('CREAM CUPID HEARTS COAT HANGER',),
 84029: ('KNITTED UNION FLAG HOT WATER BOTTLE',
  'RED WOOLLY HOTTIE WHITE HEART.'),
 82494: ('WOODEN FRAME ANTIQUE WHITE ',),
 85099: ('JUMBO BAG RED RETROSPOT',
  'JUMBO BAG STRAWBERRY',
  'JUMBO  BAG BAROQUE BLACK WHITE'),
 84997: ('CHILDRENS CUTLERY RETROSPOT RED ',
  'CHILDRENS CUTLERY POLKADOT GREEN ',
  'BLUE 3 PIECE POLKADOT CUTLERY SET',
  'CHILDRENS CUTLERY POLKADOT PINK',
  'GREEN 3 PIECE POLKADOT CUTLERY SET',
  'PINK 3 PIECE POLKADOT CUTLERY SET',
  'CHILDRENS CUTLERY POLKADOT BLUE',
  'RED 3 PIECE RETROSPOT CUTLERY SET'),
 84519: ('TOMATO CHARLIE+LOLA COASTER SET', 'CARROT CHARLIE+LOLA COASTER SET'),
 85183: ('CHARLIE & LOLA WASTEPAPER BIN FLORA',
  'CHARLIE & LOLA WASTEPAPER BIN BLUE'),
 85071: ('BLUE CHARLIE+LOLA PERSONAL DOORSIGN',
  'RED CHARLIE+LOLA PERSONAL DOORSIGN',
  'CHARLIE+LOLA MY ROOM DOOR SIGN',
  'CHARLIE+LOLA"EXTREMELY BUSY" 

In [32]:
# from our observations, 6 digit StockCodes are indeed variants of each other,
# although a 'base variant' of 5 digits does not exist.
# Let's add a new column indicating the number of variations for that product. default = 1

productVar = dict()
for i in d:
    productVar[i] = len(d[i])
#1.dictionary to store StockCode : product variations
    
pd.options.mode.chained_assignment = None

df['ProductVariations'] = 1
#2.set default as 1

for i in range(len(df)):
    if len(str(df[colName].iloc[i])) == 6:
        df['ProductVariations'].iloc[i] = productVar[int(str(df[colName].iloc[i])[:5])]
#3.for every row in dataframe, if the StockCode is 6 digits, 
#change the ProductVariations value to the corresponding value in the productVar dictionary
#(key-value pair)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550000,17850.0,United Kingdom,2
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750000,17850.0,United Kingdom,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,2
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,2
...,...,...,...,...,...,...,...,...,...
35,577129,22464,HANGING METAL HEART LANTERN,4,2011-11-17 19:52:00,0.215980,15602.0,United Kingdom,1
36,577168,M,Manual,1,2011-11-18 10:42:00,67.885711,12603.0,Germany,1
37,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,5.633640,12444.0,Norway,1
38,577696,M,Manual,1,2011-11-21 11:57:00,67.885711,16406.0,United Kingdom,1


In [33]:
# lastly, StockCodes that are less than 5 digits long, 
# i.e neither product code nor product variation

temp = df[df[colName].astype('str').str.len() < 5]
temp

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.000000,12583.0,France,1
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.500000,14527.0,United Kingdom,1
386,536403,POST,POSTAGE,1,2010-12-01 11:27:00,15.000000,12791.0,Netherlands,1
1123,536527,POST,POSTAGE,1,2010-12-01 13:04:00,18.000000,12662.0,Germany,1
1423,536540,C2,CARRIAGE,1,2010-12-01 14:05:00,50.000000,14911.0,EIRE,1
...,...,...,...,...,...,...,...,...,...
22,568384,M,Manual,1,2011-09-27 09:46:00,67.885711,12748.0,United Kingdom,1
24,571035,M,Manual,1,2011-10-13 12:50:00,67.885711,12446.0,RSA,1
28,574252,M,Manual,1,2011-11-03 13:24:00,67.885711,12437.0,France,1
36,577168,M,Manual,1,2011-11-18 10:42:00,67.885711,12603.0,Germany,1


In [34]:
temp['Description'].unique()

array(['POSTAGE', 'Discount', 'CARRIAGE', 'Manual',
       'PADS TO MATCH ALL CUSHIONS', 'DOTCOM POSTAGE', 'CRUK Commission'],
      dtype=object)

In [35]:
#these don't seem like real products, so we drop them
if colName == 'StockCode':
    df = df.drop(temp.index)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550000,17850.0,United Kingdom,2
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750000,17850.0,United Kingdom,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,2
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,2
...,...,...,...,...,...,...,...,...,...
33,575579,22437,SET OF 9 BLACK SKULL BALLOONS,20,2011-11-10 11:49:00,0.082794,13081.0,United Kingdom,1
34,575579,22089,PAPER BUNTING VINTAGE PAISLEY,24,2011-11-10 11:49:00,0.676032,13081.0,United Kingdom,1
35,577129,22464,HANGING METAL HEART LANTERN,4,2011-11-17 19:52:00,0.215980,15602.0,United Kingdom,1
37,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,5.633640,12444.0,Norway,1


## // StockCode is done. //

## ==== Description ==== 

- Description: Product (item) name. Nominal.


## // Description is done. //

## === Circling back: negative quantities & order cancellations ===

In [36]:
cancel = df[(df['InvoiceNo'].astype('str').str.contains('C'))]
orders = df[~(df['InvoiceNo'].astype('str').str.contains('C'))]

In [37]:
cancel.loc[(cancel['CustomerID'] == 17315) & (cancel['StockCode']==20979)].sort_values(by='Description')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
541717,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1


In [38]:
orders.loc[(orders['CustomerID'] == 17315) & (orders['StockCode']==20979)].sort_values(by='Description')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
433318,573911,20979,36 PENCILS TUBE RED RETROSPOT,6,2011-11-01 15:40:00,1.25,17315.0,United Kingdom,1


In [39]:
# from observation, we can try to merge cancels and orders by the following keys: StockCode+CustomerID

cancel['CancelDetails'] = cancel.apply(lambda row: str(row.StockCode) + str(row.CustomerID), axis=1)
orders['OrdersDetails'] = orders.apply(lambda row: str(row.StockCode) + str(row.CustomerID), axis=1)
#1.create new column
#2.for every row in that column, concatenate strings StockCode and CustomerID and put that as the value

In [40]:
merged = pd.merge(cancel, orders, left_on='CancelDetails', right_on='OrdersDetails', how='left')
merged

,InvoiceNo_x,StockCode_x,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID_x,Country_x,ProductVariations_x,CancelDetails,InvoiceNo_y,StockCode_y,Description_y,Quantity_y,InvoiceDate_y,UnitPrice_y,CustomerID_y,Country_y,ProductVariations_y,OrdersDetails
0,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,537195,35004C,SET OF 3 COLOURED FLYING DUCKS,12.0,2010-12-05 13:55:00,4.65,15311.0,United Kingdom,4.0,35004C15311.0
1,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,538651,35004C,SET OF 3 COLOURED FLYING DUCKS,12.0,2010-12-13 15:07:00,4.65,15311.0,United Kingdom,4.0,35004C15311.0
2,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,540157,35004C,SET OF 3 COLOURED FLYING DUCKS,12.0,2011-01-05 11:41:00,4.65,15311.0,United Kingdom,4.0,35004C15311.0
3,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,541293,35004C,SET OF 3 COLOURED FLYING DUCKS,12.0,2011-01-17 13:39:00,4.65,15311.0,United Kingdom,4.0,35004C15311.0
4,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,1,2255617548.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24249,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,36.0,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1.0,8497817315.0
24250,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,6.0,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1.0,8497817315.0
24251,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,36.0,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1.0,8497817315.0
24252,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,12.0,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1.0,8497817315.0


In [41]:
# there seems to be some cancelled orders that didn't merge with a corresponding order...
nomerge = merged[merged['OrdersDetails'].isna()]
nomerge

,InvoiceNo_x,StockCode_x,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID_x,Country_x,ProductVariations_x,CancelDetails,InvoiceNo_y,StockCode_y,Description_y,Quantity_y,InvoiceDate_y,UnitPrice_y,CustomerID_y,Country_y,ProductVariations_y,OrdersDetails
4,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,1,2255617548.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
5,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,1,2198417548.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
6,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,1,2198317548.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
7,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,1,2198017548.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
8,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548.0,United Kingdom,1,2148417548.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24035,C580989,22331,WOODLAND PARTY BAG + STICKER SET,-1,2011-12-06 16:42:00,1.65,12709.0,Germany,1,2233112709.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
24041,C581100,22946,WOODEN ADVENT CALENDAR CREAM,-1,2011-12-07 11:44:00,16.95,14534.0,United Kingdom,1,2294614534.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
24077,C581121,22526,WHEELBARROW FOR CHILDREN,-1,2011-12-07 12:28:00,12.75,14087.0,United Kingdom,1,2252614087.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
24127,C581305,22627,MINT KITCHEN SCALES,-1,2011-12-08 11:42:00,8.50,16933.0,United Kingdom,1,2262716933.0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [42]:
cancel.loc[(cancel['CustomerID'] == 17548) & (cancel['StockCode']==22556)].sort_values(by='Description')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,CancelDetails
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,1,2255617548.0


In [43]:
orders.loc[(orders['CustomerID'] == 17548)].sort_values(by='Description')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations,OrdersDetails
165028,550755,22926,IVORY GIANT GARDEN THERMOMETER,4,2011-04-20 12:01:00,5.95,17548.0,United Kingdom,1,2292617548.0
165024,550755,22585,PACK OF 6 BIRDY GIFT TAGS,24,2011-04-20 12:01:00,1.25,17548.0,United Kingdom,1,2258517548.0
165026,550755,22081,RIBBON REEL FLORA + FAUNA,10,2011-04-20 12:01:00,1.65,17548.0,United Kingdom,1,2208117548.0
165027,550755,22079,RIBBON REEL HEARTS DESIGN,10,2011-04-20 12:01:00,1.65,17548.0,United Kingdom,1,2207917548.0
165025,550755,22082,RIBBON REEL STRIPES DESIGN,10,2011-04-20 12:01:00,1.65,17548.0,United Kingdom,1,2208217548.0


In [44]:
# from the above, it seems like for those cancels that didn't match an order, the order truly doesn't exist
# e.g. for the above, customer 17548 cancelled an order for plasters, but did not order a plaster at all...
# we will drop these columns

df = df[~df['InvoiceNo'].isin(nomerge['InvoiceNo_x'].unique())]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550000,17850.0,United Kingdom,2
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750000,17850.0,United Kingdom,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,2
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390000,17850.0,United Kingdom,2
...,...,...,...,...,...,...,...,...,...
33,575579,22437,SET OF 9 BLACK SKULL BALLOONS,20,2011-11-10 11:49:00,0.082794,13081.0,United Kingdom,1
34,575579,22089,PAPER BUNTING VINTAGE PAISLEY,24,2011-11-10 11:49:00,0.676032,13081.0,United Kingdom,1
35,577129,22464,HANGING METAL HEART LANTERN,4,2011-11-17 19:52:00,0.215980,15602.0,United Kingdom,1
37,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,2011-11-18 13:23:00,5.633640,12444.0,Norway,1


In [45]:
#re-do the merging
cancel = df[(df['InvoiceNo'].astype('str').str.contains('C'))]
orders = df[~(df['InvoiceNo'].astype('str').str.contains('C'))]

cancel['CancelDetails'] = cancel.apply(lambda row: str(row.StockCode) + str(row.CustomerID), axis=1)
orders['OrdersDetails'] = orders.apply(lambda row: str(row.StockCode) + str(row.CustomerID), axis=1)

merged = pd.merge(cancel, orders, left_on='CancelDetails', right_on='OrdersDetails', how='left')
merged

,InvoiceNo_x,StockCode_x,Description_x,Quantity_x,InvoiceDate_x,UnitPrice_x,CustomerID_x,Country_x,ProductVariations_x,CancelDetails,InvoiceNo_y,StockCode_y,Description_y,Quantity_y,InvoiceDate_y,UnitPrice_y,CustomerID_y,Country_y,ProductVariations_y,OrdersDetails
0,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,537195,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-05 13:55:00,4.65,15311.0,United Kingdom,4,35004C15311.0
1,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,538651,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2010-12-13 15:07:00,4.65,15311.0,United Kingdom,4,35004C15311.0
2,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,540157,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-05 11:41:00,4.65,15311.0,United Kingdom,4,35004C15311.0
3,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,4,35004C15311.0,541293,35004C,SET OF 3 COLOURED FLYING DUCKS,12,2011-01-17 13:39:00,4.65,15311.0,United Kingdom,4,35004C15311.0
4,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.25,17897.0,United Kingdom,1,2296017897.0,536488,22960,JAM MAKING SET WITH JARS,8,2010-12-01 12:31:00,4.25,17897.0,United Kingdom,1,2296017897.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20732,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,36,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1,8497817315.0
20733,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,6,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1,8497817315.0
20734,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,36,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1,8497817315.0
20735,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,1,8497817315.0,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,12,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1,8497817315.0


In [46]:
# Get net quantity
merged['Quantity'] = merged['Quantity_x'] + merged['Quantity_y']
mergedKeys = merged['OrdersDetails'].unique()

# Tidy up the merged dataframe
merged = merged[['InvoiceNo_y', 'StockCode_y', 'Description_y', 'Quantity', 'InvoiceDate_y', 'UnitPrice_y', 'CustomerID_y', 'Country_y', 'ProductVariations_y']]
merged.columns = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'ProductVariations']
merged

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
0,537195,35004C,SET OF 3 COLOURED FLYING DUCKS,11,2010-12-05 13:55:00,4.65,15311.0,United Kingdom,4
1,538651,35004C,SET OF 3 COLOURED FLYING DUCKS,11,2010-12-13 15:07:00,4.65,15311.0,United Kingdom,4
2,540157,35004C,SET OF 3 COLOURED FLYING DUCKS,11,2011-01-05 11:41:00,4.65,15311.0,United Kingdom,4
3,541293,35004C,SET OF 3 COLOURED FLYING DUCKS,11,2011-01-17 13:39:00,4.65,15311.0,United Kingdom,4
4,536488,22960,JAM MAKING SET WITH JARS,2,2010-12-01 12:31:00,4.25,17897.0,United Kingdom,1
...,...,...,...,...,...,...,...,...,...
20732,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1
20733,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,5,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1
20734,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1
20735,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,11,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1


In [47]:
# check that total quantity cannot be less than zero
merged[merged['Quantity']<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
12,537847,22839,3 TIER CAKE TIN GREEN AND CREAM,-1,2010-12-08 15:33:00,14.95,14766.0,United Kingdom,1
94,562608,22847,BREAD BIN DINER STYLE IVORY,-3,2011-08-08 11:45:00,16.95,15502.0,United Kingdom,1
95,566243,22847,BREAD BIN DINER STYLE IVORY,-2,2011-09-11 11:57:00,16.95,15502.0,United Kingdom,1
107,562608,22847,BREAD BIN DINER STYLE IVORY,-3,2011-08-08 11:45:00,16.95,15502.0,United Kingdom,1
108,566243,22847,BREAD BIN DINER STYLE IVORY,-2,2011-09-11 11:57:00,16.95,15502.0,United Kingdom,1
...,...,...,...,...,...,...,...,...,...
20570,581143,23249,VINTAGE RED ENAMEL TRIM PLATE,-8,2011-12-07 13:46:00,1.65,17490.0,United Kingdom,1
20571,568480,23250,VINTAGE RED TRIM ENAMEL BOWL,-26,2011-09-27 11:52:00,1.25,17490.0,United Kingdom,1
20573,581143,23250,VINTAGE RED TRIM ENAMEL BOWL,-7,2011-12-07 13:46:00,1.25,17490.0,United Kingdom,1
20594,542533,22781,GUMBALL MAGAZINE RACK,-12,2011-01-28 13:18:00,6.75,16019.0,United Kingdom,1


In [48]:
# some rows are less than zero...let's see why
df.loc[(df['CustomerID'] == 14766) & (df['StockCode']==22839)].sort_values(by='Description')
# somehow, seems like they cancelled more than they bought...
# let's remove these columns in the final dataset

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
3410,C536625,22839,3 TIER CAKE TIN GREEN AND CREAM,-2,2010-12-02 10:46:00,14.95,14766.0,United Kingdom,1
19222,537847,22839,3 TIER CAKE TIN GREEN AND CREAM,1,2010-12-08 15:33:00,14.95,14766.0,United Kingdom,1


In [49]:
mergedKeys

array(['35004C15311.0', '2296017897.0', '2091414092.0', ...,
       '2314414397.0', '8497817315.0', '2097917315.0'], dtype=object)

In [50]:
df['OrderDetails'] = df.apply(lambda row: str(row.StockCode) + str(row.CustomerID), axis=1)
df = df[~df['OrderDetails'].isin(mergedKeys)]
df = pd.concat([df, merged])

In [51]:
df = df[df['Quantity']>0]

In [52]:
df['Quantity'].describe()

count    394271.000000
mean         12.591083
std          47.108602
min           1.000000
25%           2.000000
50%           5.000000
75%          12.000000
max       12540.000000
Name: Quantity, dtype: float64

In [53]:
df['UnitPrice'].describe()

count    394271.000000
mean          2.848911
std           3.817935
min           0.001000
25%           1.250000
50%           1.860000
75%           3.750000
max         295.000000
Name: UnitPrice, dtype: float64

In [54]:
df = df.drop(['OrderDetails'], axis=1)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,ProductVariations
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2
...,...,...,...,...,...,...,...,...,...
20732,548150,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-03-29 12:55:00,1.06,17315.0,United Kingdom,1
20733,550828,84978,HANGING HEART JAR T-LIGHT HOLDER,5,2011-04-21 10:19:00,1.25,17315.0,United Kingdom,1
20734,552537,84978,HANGING HEART JAR T-LIGHT HOLDER,35,2011-05-10 10:11:00,1.06,17315.0,United Kingdom,1
20735,566624,84978,HANGING HEART JAR T-LIGHT HOLDER,11,2011-09-14 09:36:00,1.25,17315.0,United Kingdom,1


In [55]:
df.to_csv('Data/uci_online_retail_cleaned.csv', index=False)